# Step 1. Project Description
The problem that we will look at in this project is the Boston house price dataset (This dataset is also used in chapter 3 of our text book).

We have download the dataset for free and placed it in the project directory with the filename “housing.csv“. You can also directly download the dataset:

https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.data 

The dataset describes 13 numerical properties of houses in Boston suburbs and is concerned with modeling the price of houses in those suburbs in thousands of dollars. As such, this is a regression predictive modeling problem. Input attributes include things like crime rate, proportion of nonretail business acres, chemical concentrations and more.

This is a well-studied problem in machine learning. It is convenient to work with because all of the input and output attributes are numerical and there are 506 instances to work with.

Reasonable performance for models evaluated using Mean Squared Error (MSE) are around 20 in squared thousands of dollars (or $4,500 if you take the square root). This is a nice target to aim for with our neural network model.


# Step 2. Develop a Baseline Neural Network Model
In this step we will create a baseline neural network model for the regression problem.

Let’s start off by including all of the functions and objects we will need for this tutorial.


In [1]:
import numpy as np
import pandas as pd
from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


Using TensorFlow backend.


In [2]:
from sklearn.metrics import mean_squared_error, make_scorer

In [3]:
mean_squared_error_modified = make_scorer(mean_squared_error, greater_is_better=False)

We can now load our dataset from a file in the local directory.

The dataset is in fact not in CSV format, the attributes are instead separated by whitespace. We can load this easily using the pandas library. We can then split the input (X) and output (Y) attributes so that they are easier to model with Keras and scikit-learn.

In [4]:
# load dataset
dataframe = pd.read_csv("housing.csv", delim_whitespace=True, header=None)
dataset = dataframe.values
#test_data = dataset[:20,]
#dataset = dataset[20:,]


In [5]:
# split into input (X) and output (Y) variables
X = dataset[:,0:13]
Y = dataset[:,13]
#test_x = test_data[:,0:13]
#test_y = test_data[:,13]


In [6]:
# STandarized dataset
standarized = StandardScaler()
standarized.fit(X)
X_standarized = standarized.transform(X)

In [7]:
X_standarized

array([[-0.41978194,  0.28482986, -1.2879095 , ..., -1.45900038,
         0.44105193, -1.0755623 ],
       [-0.41733926, -0.48772236, -0.59338101, ..., -0.30309415,
         0.44105193, -0.49243937],
       [-0.41734159, -0.48772236, -0.59338101, ..., -0.30309415,
         0.39642699, -1.2087274 ],
       ...,
       [-0.41344658, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.98304761],
       [-0.40776407, -0.48772236,  0.11573841, ...,  1.17646583,
         0.4032249 , -0.86530163],
       [-0.41500016, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.66905833]])

In [8]:
#normalized data set
X_normalized = X/X.max(axis = 0)

In [9]:
X_normalized

array([[7.10302306e-05, 1.80000000e-01, 8.32732516e-02, ...,
        6.95454545e-01, 1.00000000e+00, 1.31156176e-01],
       [3.06936012e-04, 0.00000000e+00, 2.54866619e-01, ...,
        8.09090909e-01, 1.00000000e+00, 2.40716355e-01],
       [3.06711233e-04, 0.00000000e+00, 2.54866619e-01, ...,
        8.09090909e-01, 9.89745528e-01, 1.06136423e-01],
       ...,
       [6.82879242e-04, 0.00000000e+00, 4.30064888e-01, ...,
        9.54545455e-01, 1.00000000e+00, 1.48538320e-01],
       [1.23167768e-03, 0.00000000e+00, 4.30064888e-01, ...,
        9.54545455e-01, 9.91307634e-01, 1.70661048e-01],
       [5.32839119e-04, 0.00000000e+00, 4.30064888e-01, ...,
        9.54545455e-01, 1.00000000e+00, 2.07532262e-01]])

We can create Keras models and evaluate them with scikit-learn by using handy wrapper objects provided by the Keras library. This is desirable, because scikit-learn excels at evaluating models and will allow us to use powerful data preparation and model evaluation schemes with very few lines of code.

The Keras wrappers require a function as an argument. This function that we must define is responsible for creating the neural network model to be evaluated.

Below we define the function to create the baseline model to be evaluated. It is a simple model that has a single fully connected hidden layer with the same number of neurons as input attributes (13). The network uses good practices such as the rectifier activation function for the hidden layer. No activation function is used for the output layer because it is a regression problem and we are interested in predicting numerical values directly without transform.

The efficient ADAM optimization algorithm is used and a mean squared error loss function is optimized. This will be the same metric that we will use to evaluate the performance of the model. It is a desirable metric because by taking the square root gives us an error value we can directly understand in the context of the problem (thousands of dollars).


In [208]:
# define base model
def baseline_model():
    model = models.Sequential()
    model.add(layers.Dense(13, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    return model


The Keras wrapper object for use in scikit-learn as a regression estimator is called KerasRegressor. We create an instance and pass it both the name of the function to create the neural network model as well as some parameters to pass along to the fit() function of the model later, such as the number of epochs and batch size. Both of these are set to sensible defaults.

We also initialize the random number generator with a constant random seed, a process we will repeat for each model evaluated in this tutorial. This is an attempt to ensure we compare models consistently.


In [209]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Results: -32.82 (23.28) MSE


The final step is to evaluate this baseline model. We will use 10-fold cross validation to evaluate the model.Running this code gives us an estimate of the model’s performance on the problem for unseen data. The result reports the mean squared error including the average and standard deviation (average variance) across all 10 folds of the cross validation evaluation.

# Step 3: Modeling The Standardized Dataset
An important concern with the Boston house price dataset is that the input attributes all vary in their scales because they measure different quantities.

It is almost always good practice to prepare your data before modeling it using a neural network model.

Continuing on from the above baseline model, we can re-evaluate the same model using a standardized version of the input dataset.

We can use scikit-learn’s Pipeline framework to perform the standardization during the model evaluation process, within each fold of the cross validation. This ensures that there is no data leakage from each testset cross validation fold into the training data:

http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html


The code below creates a scikit-learn Pipeline that first standardizes the dataset then creates and evaluates the baseline neural network model.
Running the code provides an improved performance over the baseline model without standardized data, dropping the error.

In [210]:
# evaluate model with standardized dataset
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))


Standardized: -29.59 (27.57) MSE


# Extension of Step 3:
A further extension of this step is to similarly apply a rescaling to the output variable such as normalizing it to the range of 0-1 and use a Sigmoid or similar activation function on the output layer to narrow output predictions to the same range.


In [211]:
def baseline_model_normalized():
    model = models.Sequential()
    model.add(layers.Dense(13, activation = 'sigmoid', input_shape = (X.shape[1],)))
    model.add(layers.Dense(1))
    model.compile(optimizer = 'adam', loss = 'mse')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model_normalized, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X_normalized, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Results: -51.91 (37.66) MSE


# Step 4. Tune The Neural Network Topology
There are many concerns that can be optimized for a neural network model.

Perhaps the point of biggest leverage is the structure of the network itself, including the number of layers and the number of neurons in each layer.

In this section we will evaluate two additional network topologies in an effort to further improve the performance of the model. We will look at both a deeper and a wider network topology.


## Step 4.1. Evaluate a Deeper Network Topology
One way to improve the performance of a neural network is to add more layers. This might allow the model to extract and recombine higher order features embedded in the data.

In this step we will evaluate the effect of adding one more hidden layer to the model. This is as easy as defining a new function that will create this deeper model, copied from our baseline model above. We can then insert a new line after the first hidden layer. In this case with about half the number of neurons.


Our network topology now looks like:

13 inputs -> [13 -> 6] -> 1 output

We can evaluate this network topology in the same way as above, whilst also using the standardization of the dataset that above was shown to improve performance.



In [212]:
# define the model
def larger_model():
    model = models.Sequential()
    model.add(layers.Dense(13, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dense(6, activation = 'relu',kernel_initializer = 'normal'))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))


Larger: -23.04 (27.04) MSE


## Step 4.2. Evaluate a Wider Network Topology
Another approach to increasing the representational capability of the model is to create a wider network.

In this section we evaluate the effect of keeping a shallow network architecture and nearly doubling the number of neurons in the one hidden layer.

Again, all we need to do is define a new function that creates our neural network model. Here, we have increased the number of neurons in the hidden layer compared to the baseline model from 13 to 20.


In [214]:
# define the model
def wider_model():
    model = models.Sequential()
    model.add(layers.Dense(20, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -22.81 (29.18) MSE


# Step 5. Really Scaling up: developing a model that overfits
Once you’ve obtained a model that has statistical power, the question becomes, is your
model sufficiently powerful? Does it have enough layers and parameters to properly
model the problem at hand? 

Remember that the universal tension in machine learning is between optimization and generalization; the ideal model is one that stands right at the border between underfitting and overfitting; between undercapacity and overcapacity. To figure out where this border lies, first you must cross it.

To figure out how big a model you’ll need, you must develop a model that overfits.
This is fairly easy:
1.	Add layers.
2.	Make the layers bigger.
3.	Train for more epochs.

Always monitor the training loss and validation loss, as well as the training and validation values for any metrics you care about. When you see that the model’s performance on the validation data begins to degrade, you’ve achieved overfitting.

The next step is to start regularizing and tuning the model, to get as close as possible to the ideal model that neither underfits nor overfits.


In [10]:
# model with a more neurons
def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=100, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/100
455/455 [==============================] - 0s 739us/step - loss: 595.1686
Epoch 2/100
455/455 [==============================] - 0s 110us/step - loss: 530.7832
Epoch 3/100
455/455 [==============================] - 0s 116us/step - loss: 396.0766
Epoch 4/100
455/455 [==============================] - 0s 114us/step - loss: 239.5082
Epoch 5/100
455/455 [==============================] - 0s 110us/step - loss: 125.6734
Epoch 6/100
455/455 [==============================] - 0s 110us/step - loss: 71.3354
Epoch 7/100
455/455 [==============================] - 0s 164us/step - loss: 49.7038
Epoch 8/100
455/455 [==============================] - 0s 122us/step - loss: 39.7463
Epoch 9/100
455/455 [==============================] - 0s 118us/step - loss: 34.0692
Epoch 10/100
455/455 [==============================] - 0s 136us/step - loss: 30.7148
Epoch 11/100
455/455 [==============================] - 0s 112us/step - loss: 28.5647
Epoch 12/100
455/455 [==============================] - 0s

455/455 [==============================] - 0s 101us/step - loss: 10.5400
Epoch 91/100
455/455 [==============================] - 0s 112us/step - loss: 10.4439
Epoch 92/100
455/455 [==============================] - 0s 98us/step - loss: 10.3849
Epoch 93/100
455/455 [==============================] - 0s 87us/step - loss: 10.4131
Epoch 94/100
455/455 [==============================] - 0s 107us/step - loss: 10.3787
Epoch 95/100
455/455 [==============================] - 0s 94us/step - loss: 10.3523
Epoch 96/100
455/455 [==============================] - 0s 94us/step - loss: 10.3392
Epoch 97/100
455/455 [==============================] - 0s 112us/step - loss: 10.3711
Epoch 98/100
455/455 [==============================] - 0s 125us/step - loss: 10.2552
Epoch 99/100
455/455 [==============================] - 0s 114us/step - loss: 10.2364
Epoch 100/100
51/51 [==============================] - 0s 404us/step
Epoch 1/100
455/455 [==============================] - 0s 371us/step - loss: 608.5930
Ep

455/455 [==============================] - 0s 105us/step - loss: 10.0816
Epoch 86/100
455/455 [==============================] - 0s 142us/step - loss: 9.9929
Epoch 87/100
455/455 [==============================] - 0s 118us/step - loss: 10.1057
Epoch 88/100
455/455 [==============================] - 0s 112us/step - loss: 9.9584
Epoch 89/100
455/455 [==============================] - 0s 112us/step - loss: 9.9690
Epoch 90/100
455/455 [==============================] - 0s 118us/step - loss: 9.9225
Epoch 91/100
455/455 [==============================] - 0s 116us/step - loss: 9.9037
Epoch 92/100
455/455 [==============================] - 0s 112us/step - loss: 9.9046
Epoch 93/100
455/455 [==============================] - 0s 110us/step - loss: 9.9328
Epoch 94/100
455/455 [==============================] - 0s 107us/step - loss: 9.9142
Epoch 95/100
455/455 [==============================] - 0s 101us/step - loss: 9.8200
Epoch 96/100
455/455 [==============================] - 0s 103us/step - loss

455/455 [==============================] - 0s 142us/step - loss: 10.6810
Epoch 75/100
455/455 [==============================] - 0s 142us/step - loss: 10.7206
Epoch 76/100
455/455 [==============================] - 0s 107us/step - loss: 10.6705
Epoch 77/100
455/455 [==============================] - 0s 166us/step - loss: 10.5531
Epoch 78/100
455/455 [==============================] - 0s 133us/step - loss: 10.5058
Epoch 79/100
455/455 [==============================] - 0s 127us/step - loss: 10.4549
Epoch 80/100
455/455 [==============================] - 0s 136us/step - loss: 10.4412
Epoch 81/100
455/455 [==============================] - 0s 132us/step - loss: 10.3103
Epoch 82/100
455/455 [==============================] - 0s 134us/step - loss: 10.3055
Epoch 83/100
455/455 [==============================] - 0s 140us/step - loss: 10.2448
Epoch 84/100
455/455 [==============================] - 0s 111us/step - loss: 10.2820
Epoch 85/100
455/455 [==============================] - 0s 170us/st

Epoch 69/100
455/455 [==============================] - 0s 144us/step - loss: 10.8660
Epoch 70/100
455/455 [==============================] - 0s 142us/step - loss: 10.6938
Epoch 71/100
455/455 [==============================] - 0s 145us/step - loss: 10.7761
Epoch 72/100
455/455 [==============================] - 0s 140us/step - loss: 10.7036
Epoch 73/100
455/455 [==============================] - 0s 138us/step - loss: 10.6143
Epoch 74/100
455/455 [==============================] - 0s 139us/step - loss: 10.5321
Epoch 75/100
455/455 [==============================] - 0s 99us/step - loss: 10.5554
Epoch 76/100
455/455 [==============================] - 0s 138us/step - loss: 10.4305
Epoch 77/100
455/455 [==============================] - 0s 136us/step - loss: 10.4059
Epoch 78/100
455/455 [==============================] - 0s 135us/step - loss: 10.4621
Epoch 79/100
455/455 [==============================] - 0s 136us/step - loss: 10.2496
Epoch 80/100
455/455 [==============================] -

456/456 [==============================] - 0s 157us/step - loss: 7.2671
Epoch 59/100
456/456 [==============================] - 0s 147us/step - loss: 7.1677
Epoch 60/100
456/456 [==============================] - 0s 146us/step - loss: 7.0378
Epoch 61/100
456/456 [==============================] - 0s 142us/step - loss: 7.0314
Epoch 62/100
456/456 [==============================] - 0s 143us/step - loss: 6.9246
Epoch 63/100
456/456 [==============================] - 0s 142us/step - loss: 6.9233
Epoch 64/100
456/456 [==============================] - 0s 151us/step - loss: 6.8661
Epoch 65/100
456/456 [==============================] - 0s 151us/step - loss: 6.7681
Epoch 66/100
456/456 [==============================] - 0s 151us/step - loss: 6.7720
Epoch 67/100
456/456 [==============================] - 0s 160us/step - loss: 6.7275
Epoch 68/100
456/456 [==============================] - 0s 149us/step - loss: 6.5684
Epoch 69/100
456/456 [==============================] - 0s 147us/step - loss: 

456/456 [==============================] - 0s 158us/step - loss: 14.2011
Epoch 49/100
456/456 [==============================] - 0s 157us/step - loss: 14.0393
Epoch 50/100
456/456 [==============================] - 0s 155us/step - loss: 13.8960
Epoch 51/100
456/456 [==============================] - 0s 155us/step - loss: 13.5975
Epoch 52/100
456/456 [==============================] - 0s 160us/step - loss: 13.5714
Epoch 53/100
456/456 [==============================] - 0s 160us/step - loss: 13.4160
Epoch 54/100
456/456 [==============================] - 0s 160us/step - loss: 13.1465
Epoch 55/100
456/456 [==============================] - 0s 155us/step - loss: 12.9686
Epoch 56/100
456/456 [==============================] - 0s 157us/step - loss: 12.8566
Epoch 57/100
456/456 [==============================] - 0s 153us/step - loss: 12.7388
Epoch 58/100
456/456 [==============================] - 0s 142us/step - loss: 12.5541
Epoch 59/100
456/456 [==============================] - 0s 149us/st

In [11]:
# model with a more neurons
def model2():
    model = models.Sequential()
    model.add(layers.Dense(30, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model2, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -22.28 (22.63) MSE


Since performance started to degrade so going with model 1

# Step 6. Tuning the Model
With further tuning of aspects like the optimization algorithm etc. and the number of training epochs, it is expected that further improvements are possible. What is the best score that you can achieve on this dataset?


In [13]:
# model1 with rmsprop

def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -20.94 (24.39) MSE


In [14]:
# model1 with rmsprop and weight drop of 20%

def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -22.52 (25.72) MSE


In [16]:
# model1 with rmsprop and weight drop of 30%

def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -23.58 (25.82) MSE


In [17]:
# model1 with rmsprop and weight drop of 50%

def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -27.27 (30.01) MSE


In [19]:
# model1 with rmsprop and weight regularizer
from keras import regularizers

def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal',
                           kernel_regularizer = regularizers.l1(0.001),
                           input_shape = (X.shape[1],)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -26.97 (29.64) MSE


Since droping weight is not producing good results so using wieght regularizer

In [20]:
# model1 with rmsprop and weight regularizer l1
from keras import regularizers

def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal',
                           kernel_regularizer = regularizers.l1(0.001),
                           input_shape = (X.shape[1],)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -27.07 (29.93) MSE


In [21]:
# model1 with rmsprop and weight regularizer l2
from keras import regularizers

def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal',
                           kernel_regularizer = regularizers.l2(0.001),
                           input_shape = (X.shape[1],)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -26.60 (29.87) MSE


In [22]:
# model1 with rmsprop and weight drop of 20%

def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=150, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -21.82 (27.15) MSE


In [23]:
# model1 with rmsprop with 110 epochs

def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=110, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)

In [24]:
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -20.99 (25.98) MSE


In [26]:
# model1 with rmsprop with 150 epochs

def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=150, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -22.55 (27.71) MSE


In [27]:

# model1 with rmsprop with 130 epochs

def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=130, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -21.26 (26.61) MSE


In [28]:
# model1 with rmsprop with 120 epochs

def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=120, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -21.41 (26.81) MSE


In [29]:
# model1 with rmsprop with 115 epochs

def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=115, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -21.16 (25.63) MSE


In [30]:
# model1 with rmsprop with 112 epochs

def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=112, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -22.15 (28.91) MSE


In [33]:
# model1 with rmsprop with 110 epochs and second layer of 10 neurons

def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dense(10, activation = 'relu',kernel_initializer = 'normal'))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=110, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -20.52 (23.56) MSE


In [34]:
# model1 with rmsprop with 110 epochs and second layer of 13 neurons

def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dense(13, activation = 'relu',kernel_initializer = 'normal'))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=110, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -20.62 (25.30) MSE


In [35]:
# model1 with rmsprop with 110 epochs and second layer of 7 neurons

def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dense(7, activation = 'relu',kernel_initializer = 'normal'))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=110, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -20.33 (24.75) MSE


In [36]:
# model1 with rmsprop with 110 epochs and second layer of 5 neurons

def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dense(5, activation = 'relu',kernel_initializer = 'normal'))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=110, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -20.74 (25.62) MSE


In [37]:
# model1 with rmsprop with 110 epochs and second layer of 7 neurons and second layer with 3 neurons

def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dense(7, activation = 'relu',kernel_initializer = 'normal'))
    model.add(layers.Dense(3, activation = 'relu',kernel_initializer = 'normal'))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=110, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -29.45 (34.63) MSE


In [53]:
# model1 with rmsprop with 110 epochs, and second layer of 7 neurons, changing batch size to 1

def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dense(7, activation = 'relu',kernel_initializer = 'normal'))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=110, batch_size=1, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/110
455/455 [==============================] - 5s 11ms/step - loss: 423.4692
Epoch 2/110
455/455 [==============================] - 1s 3ms/step - loss: 62.3609
Epoch 3/110
455/455 [==============================] - 1s 3ms/step - loss: 36.0764
Epoch 4/110
455/455 [==============================] - 1s 3ms/step - loss: 28.8702
Epoch 5/110
455/455 [==============================] - 1s 3ms/step - loss: 25.5660
Epoch 6/110
455/455 [==============================] - 2s 3ms/step - loss: 23.4617
Epoch 7/110
455/455 [==============================] - 1s 3ms/step - loss: 21.8758
Epoch 8/110
455/455 [==============================] - 2s 3ms/step - loss: 20.5043
Epoch 9/110
455/455 [==============================] - 2s 3ms/step - loss: 19.7998
Epoch 10/110
455/455 [==============================] - 1s 3ms/step - loss: 18.5868
Epoch 11/110
455/455 [==============================] - 2s 3ms/step - loss: 17.9350
Epoch 12/110
455/455 [==============================] - 1s 3ms/step - loss: 17.2447

Epoch 99/110
455/455 [==============================] - 1s 3ms/step - loss: 9.4764
Epoch 100/110
455/455 [==============================] - 1s 3ms/step - loss: 9.4119
Epoch 101/110
455/455 [==============================] - 1s 3ms/step - loss: 9.3769
Epoch 102/110
455/455 [==============================] - 1s 3ms/step - loss: 9.1168
Epoch 103/110
455/455 [==============================] - 1s 3ms/step - loss: 9.1232
Epoch 104/110
455/455 [==============================] - 1s 3ms/step - loss: 9.3250
Epoch 105/110
455/455 [==============================] - 1s 3ms/step - loss: 9.3871
Epoch 106/110
455/455 [==============================] - 1s 3ms/step - loss: 9.1520
Epoch 107/110
455/455 [==============================] - 1s 3ms/step - loss: 9.3408
Epoch 108/110
455/455 [==============================] - 1s 3ms/step - loss: 9.2673
Epoch 109/110
455/455 [==============================] - 1s 3ms/step - loss: 9.1001
Epoch 110/110
51/51 [==============================] - 2s 44ms/step
Epoch 1/1

455/455 [==============================] - 1s 3ms/step - loss: 10.4782
Epoch 73/110
455/455 [==============================] - 1s 3ms/step - loss: 10.6341
Epoch 74/110
455/455 [==============================] - 1s 3ms/step - loss: 10.5748
Epoch 75/110
455/455 [==============================] - 1s 3ms/step - loss: 10.6557
Epoch 76/110
455/455 [==============================] - 1s 3ms/step - loss: 10.4427
Epoch 77/110
455/455 [==============================] - 1s 3ms/step - loss: 10.4669
Epoch 78/110
455/455 [==============================] - 1s 3ms/step - loss: 10.2237
Epoch 79/110
455/455 [==============================] - 1s 3ms/step - loss: 10.4587
Epoch 80/110
455/455 [==============================] - 1s 3ms/step - loss: 10.3920
Epoch 81/110
455/455 [==============================] - 1s 3ms/step - loss: 10.3856
Epoch 82/110
455/455 [==============================] - 1s 3ms/step - loss: 10.3992
Epoch 83/110
455/455 [==============================] - 1s 3ms/step - loss: 10.4829
Epoch

455/455 [==============================] - 1s 3ms/step - loss: 9.9119
Epoch 60/110
455/455 [==============================] - 1s 3ms/step - loss: 9.9235
Epoch 61/110
455/455 [==============================] - 1s 3ms/step - loss: 9.9858
Epoch 62/110
455/455 [==============================] - 1s 3ms/step - loss: 9.9031
Epoch 63/110
455/455 [==============================] - 1s 3ms/step - loss: 9.8441
Epoch 64/110
455/455 [==============================] - 1s 3ms/step - loss: 9.6669
Epoch 65/110
455/455 [==============================] - 1s 3ms/step - loss: 9.8943
Epoch 66/110
455/455 [==============================] - 1s 3ms/step - loss: 9.4481
Epoch 67/110
455/455 [==============================] - 1s 3ms/step - loss: 9.3877
Epoch 68/110
455/455 [==============================] - 1s 3ms/step - loss: 9.5895
Epoch 69/110
455/455 [==============================] - 1s 3ms/step - loss: 9.4487
Epoch 70/110
455/455 [==============================] - 1s 3ms/step - loss: 9.3908
Epoch 71/110
455/

455/455 [==============================] - 2s 3ms/step - loss: 11.5938
Epoch 34/110
455/455 [==============================] - 1s 3ms/step - loss: 11.7341
Epoch 35/110
455/455 [==============================] - 1s 3ms/step - loss: 11.5748
Epoch 36/110
455/455 [==============================] - 1s 3ms/step - loss: 11.5928
Epoch 37/110
455/455 [==============================] - 1s 3ms/step - loss: 11.6664
Epoch 38/110
455/455 [==============================] - 2s 3ms/step - loss: 11.4924
Epoch 39/110
455/455 [==============================] - 2s 4ms/step - loss: 11.5115
Epoch 40/110
455/455 [==============================] - 2s 4ms/step - loss: 11.4114
Epoch 41/110
455/455 [==============================] - 2s 4ms/step - loss: 11.2736
Epoch 42/110
455/455 [==============================] - 1s 3ms/step - loss: 11.1230
Epoch 43/110
455/455 [==============================] - 2s 4ms/step - loss: 11.2256
Epoch 44/110
455/455 [==============================] - 2s 3ms/step - loss: 11.0238
Epoch

456/456 [==============================] - 1s 3ms/step - loss: 16.4568
Epoch 8/110
456/456 [==============================] - 1s 3ms/step - loss: 15.7788
Epoch 9/110
456/456 [==============================] - 1s 3ms/step - loss: 14.9006
Epoch 10/110
456/456 [==============================] - 1s 3ms/step - loss: 14.0060
Epoch 11/110
456/456 [==============================] - 1s 3ms/step - loss: 13.1821
Epoch 12/110
456/456 [==============================] - 1s 3ms/step - loss: 12.1454
Epoch 13/110
456/456 [==============================] - 1s 3ms/step - loss: 11.5447
Epoch 14/110
456/456 [==============================] - 1s 3ms/step - loss: 11.1937
Epoch 15/110
456/456 [==============================] - 1s 3ms/step - loss: 10.6554
Epoch 16/110
456/456 [==============================] - 1s 3ms/step - loss: 9.8969
Epoch 17/110
456/456 [==============================] - 1s 3ms/step - loss: 9.7988
Epoch 18/110
456/456 [==============================] - 2s 4ms/step - loss: 9.0761
Epoch 19/1

456/456 [==============================] - 1s 3ms/step - loss: 9.3237
Epoch 93/110
456/456 [==============================] - 1s 3ms/step - loss: 9.2126
Epoch 94/110
456/456 [==============================] - 1s 3ms/step - loss: 9.3905
Epoch 95/110
456/456 [==============================] - 1s 3ms/step - loss: 9.1329
Epoch 96/110
456/456 [==============================] - 1s 3ms/step - loss: 9.1362
Epoch 97/110
456/456 [==============================] - 1s 3ms/step - loss: 9.1785
Epoch 98/110
456/456 [==============================] - 1s 3ms/step - loss: 9.1884
Epoch 99/110
456/456 [==============================] - 2s 3ms/step - loss: 9.3988
Epoch 100/110
456/456 [==============================] - 1s 3ms/step - loss: 9.2949
Epoch 101/110
456/456 [==============================] - 1s 3ms/step - loss: 9.3759
Epoch 102/110
456/456 [==============================] - 1s 3ms/step - loss: 9.4444
Epoch 103/110
456/456 [==============================] - 2s 3ms/step - loss: 9.2719
Epoch 104/110

In [54]:
# model1 with rmsprop with 110 epochs, and second layer of 7 neurons, changing batch size to 10

def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dense(7, activation = 'relu',kernel_initializer = 'normal'))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=110, batch_size=10, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/110
455/455 [==============================] - 5s 10ms/step - loss: 601.1684
Epoch 2/110
455/455 [==============================] - 0s 480us/step - loss: 573.9994
Epoch 3/110
455/455 [==============================] - 0s 436us/step - loss: 516.1603
Epoch 4/110
455/455 [==============================] - 0s 316us/step - loss: 427.7817
Epoch 5/110
455/455 [==============================] - 0s 327us/step - loss: 319.2419
Epoch 6/110
455/455 [==============================] - 0s 362us/step - loss: 206.1201
Epoch 7/110
455/455 [==============================] - 0s 322us/step - loss: 121.3474
Epoch 8/110
455/455 [==============================] - 0s 519us/step - loss: 78.4065
Epoch 9/110
455/455 [==============================] - 0s 438us/step - loss: 59.4899
Epoch 10/110
455/455 [==============================] - 0s 453us/step - loss: 47.0909
Epoch 11/110
455/455 [==============================] - 0s 550us/step - loss: 39.1886
Epoch 12/110
455/455 [==============================] - 0

455/455 [==============================] - 0s 530us/step - loss: 9.6639
Epoch 97/110
455/455 [==============================] - 0s 590us/step - loss: 9.4847
Epoch 98/110
455/455 [==============================] - 0s 586us/step - loss: 9.5191
Epoch 99/110
455/455 [==============================] - 0s 487us/step - loss: 9.5015
Epoch 100/110
455/455 [==============================] - 0s 581us/step - loss: 9.4832
Epoch 101/110
455/455 [==============================] - 0s 509us/step - loss: 9.3777
Epoch 102/110
455/455 [==============================] - 0s 612us/step - loss: 9.3909
Epoch 103/110
455/455 [==============================] - 0s 438us/step - loss: 9.2213
Epoch 104/110
455/455 [==============================] - 0s 515us/step - loss: 9.4056
Epoch 105/110
455/455 [==============================] - 0s 545us/step - loss: 9.3128
Epoch 106/110
455/455 [==============================] - 0s 433us/step - loss: 9.2387
Epoch 107/110
455/455 [==============================] - 0s 401us/step 

455/455 [==============================] - 0s 331us/step - loss: 12.0641
Epoch 65/110
455/455 [==============================] - 0s 393us/step - loss: 11.9867
Epoch 66/110
455/455 [==============================] - 0s 416us/step - loss: 11.9160
Epoch 67/110
455/455 [==============================] - 0s 342us/step - loss: 11.6929
Epoch 68/110
455/455 [==============================] - 0s 311us/step - loss: 11.7248
Epoch 69/110
455/455 [==============================] - 0s 340us/step - loss: 11.7554
Epoch 70/110
455/455 [==============================] - 0s 317us/step - loss: 11.5720
Epoch 71/110
455/455 [==============================] - 0s 324us/step - loss: 11.6605
Epoch 72/110
455/455 [==============================] - 0s 322us/step - loss: 11.5416
Epoch 73/110
455/455 [==============================] - 0s 320us/step - loss: 11.4723
Epoch 74/110
455/455 [==============================] - 0s 319us/step - loss: 11.4014
Epoch 75/110
455/455 [==============================] - 0s 331us/st

455/455 [==============================] - 0s 285us/step - loss: 19.3511
Epoch 33/110
455/455 [==============================] - 0s 295us/step - loss: 19.0447
Epoch 34/110
455/455 [==============================] - 0s 290us/step - loss: 18.6879
Epoch 35/110
455/455 [==============================] - 0s 287us/step - loss: 18.4654
Epoch 36/110
455/455 [==============================] - 0s 287us/step - loss: 18.0226
Epoch 37/110
455/455 [==============================] - 0s 292us/step - loss: 17.8036
Epoch 38/110
455/455 [==============================] - 0s 287us/step - loss: 17.4416
Epoch 39/110
455/455 [==============================] - 0s 313us/step - loss: 17.2190
Epoch 40/110
455/455 [==============================] - 0s 386us/step - loss: 16.9479
Epoch 41/110
455/455 [==============================] - 0s 373us/step - loss: 16.7174
Epoch 42/110
455/455 [==============================] - 0s 318us/step - loss: 16.4622
Epoch 43/110
455/455 [==============================] - 0s 303us/st

51/51 [==============================] - 2s 44ms/step
Epoch 1/110
456/456 [==============================] - 6s 12ms/step - loss: 596.8739
Epoch 2/110
456/456 [==============================] - 0s 335us/step - loss: 567.8636
Epoch 3/110
456/456 [==============================] - 0s 330us/step - loss: 504.4702
Epoch 4/110
456/456 [==============================] - 0s 332us/step - loss: 402.8458
Epoch 5/110
456/456 [==============================] - 0s 330us/step - loss: 281.6387
Epoch 6/110
456/456 [==============================] - 0s 321us/step - loss: 167.9426
Epoch 7/110
456/456 [==============================] - 0s 336us/step - loss: 98.3881
Epoch 8/110
456/456 [==============================] - 0s 331us/step - loss: 69.3095
Epoch 9/110
456/456 [==============================] - 0s 328us/step - loss: 55.5402
Epoch 10/110
456/456 [==============================] - 0s 330us/step - loss: 45.7042
Epoch 11/110
456/456 [==============================] - 0s 337us/step - loss: 39.1954
Epoc

456/456 [==============================] - 0s 339us/step - loss: 6.8374
Epoch 80/110
456/456 [==============================] - 0s 350us/step - loss: 6.8073
Epoch 81/110
456/456 [==============================] - 0s 329us/step - loss: 6.7916
Epoch 82/110
456/456 [==============================] - 0s 330us/step - loss: 6.7851
Epoch 83/110
456/456 [==============================] - 0s 324us/step - loss: 6.6654
Epoch 84/110
456/456 [==============================] - 0s 332us/step - loss: 6.6519
Epoch 85/110
456/456 [==============================] - 0s 334us/step - loss: 6.6440
Epoch 86/110
456/456 [==============================] - 0s 348us/step - loss: 6.5718
Epoch 87/110
456/456 [==============================] - 0s 328us/step - loss: 6.6168
Epoch 88/110
456/456 [==============================] - 0s 379us/step - loss: 6.5132
Epoch 89/110
456/456 [==============================] - 0s 410us/step - loss: 6.4776
Epoch 90/110
456/456 [==============================] - 0s 380us/step - loss: 

456/456 [==============================] - 0s 333us/step - loss: 10.7235
Epoch 65/110
456/456 [==============================] - 0s 317us/step - loss: 10.6292
Epoch 66/110
456/456 [==============================] - 0s 333us/step - loss: 10.4044
Epoch 67/110
456/456 [==============================] - 0s 335us/step - loss: 10.4879
Epoch 68/110
456/456 [==============================] - 0s 317us/step - loss: 10.3207
Epoch 69/110
456/456 [==============================] - 0s 324us/step - loss: 10.3148
Epoch 70/110
456/456 [==============================] - 0s 337us/step - loss: 10.1562
Epoch 71/110
456/456 [==============================] - 0s 328us/step - loss: 10.1332
Epoch 72/110
456/456 [==============================] - 0s 330us/step - loss: 10.0547
Epoch 73/110
456/456 [==============================] - 0s 324us/step - loss: 9.9504
Epoch 74/110
456/456 [==============================] - 0s 328us/step - loss: 9.8814
Epoch 75/110
456/456 [==============================] - 0s 329us/step

456/456 [==============================] - 0s 304us/step - loss: 15.0114
Epoch 50/110
456/456 [==============================] - 0s 304us/step - loss: 14.8900
Epoch 51/110
456/456 [==============================] - 0s 306us/step - loss: 14.7332
Epoch 52/110
456/456 [==============================] - 0s 409us/step - loss: 14.5275
Epoch 53/110
456/456 [==============================] - 0s 305us/step - loss: 14.3360
Epoch 54/110
456/456 [==============================] - 0s 305us/step - loss: 14.1198
Epoch 55/110
456/456 [==============================] - 0s 300us/step - loss: 13.9826
Epoch 56/110
456/456 [==============================] - 0s 294us/step - loss: 13.8279
Epoch 57/110
456/456 [==============================] - 0s 356us/step - loss: 13.5933
Epoch 58/110
456/456 [==============================] - 0s 375us/step - loss: 13.5299
Epoch 59/110
456/456 [==============================] - 0s 317us/step - loss: 13.2035
Epoch 60/110
456/456 [==============================] - 0s 295us/st

In [55]:
# model1 with rmsprop with 110 epochs, and second layer of 7 neurons, changing batch size to 7

def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dense(7, activation = 'relu',kernel_initializer = 'normal'))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return model
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model1, epochs=110, batch_size=7, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/110
455/455 [==============================] - 7s 14ms/step - loss: 597.3289
Epoch 2/110
455/455 [==============================] - 0s 932us/step - loss: 547.3509
Epoch 3/110
455/455 [==============================] - 0s 962us/step - loss: 440.1749
Epoch 4/110
455/455 [==============================] - 0s 662us/step - loss: 290.0866
Epoch 5/110
455/455 [==============================] - 0s 678us/step - loss: 151.7858
Epoch 6/110
455/455 [==============================] - 0s 813us/step - loss: 75.9759
Epoch 7/110
455/455 [==============================] - 0s 809us/step - loss: 52.8096
Epoch 8/110
455/455 [==============================] - 0s 923us/step - loss: 40.4388
Epoch 9/110
455/455 [==============================] - 0s 854us/step - loss: 33.5877
Epoch 10/110
455/455 [==============================] - 0s 1ms/step - loss: 29.8169
Epoch 11/110
455/455 [==============================] - 0s 1ms/step - loss: 27.7314
Epoch 12/110
455/455 [==============================] - 0s 1ms/

Epoch 81/110
455/455 [==============================] - 0s 343us/step - loss: 10.6099
Epoch 82/110
455/455 [==============================] - 0s 309us/step - loss: 10.6471
Epoch 83/110
455/455 [==============================] - 0s 343us/step - loss: 10.5409
Epoch 84/110
455/455 [==============================] - 0s 343us/step - loss: 10.5671
Epoch 85/110
455/455 [==============================] - 0s 363us/step - loss: 10.5814
Epoch 86/110
455/455 [==============================] - 0s 424us/step - loss: 10.4956
Epoch 87/110
455/455 [==============================] - 0s 431us/step - loss: 10.3429
Epoch 88/110
455/455 [==============================] - 0s 333us/step - loss: 10.4242
Epoch 89/110
455/455 [==============================] - 0s 336us/step - loss: 10.4145
Epoch 90/110
455/455 [==============================] - 0s 343us/step - loss: 10.4678
Epoch 91/110
455/455 [==============================] - 0s 322us/step - loss: 10.3169
Epoch 92/110
455/455 [==============================] 

455/455 [==============================] - 0s 765us/step - loss: 12.1339
Epoch 50/110
455/455 [==============================] - 0s 738us/step - loss: 12.0977
Epoch 51/110
455/455 [==============================] - 0s 739us/step - loss: 11.9972
Epoch 52/110
455/455 [==============================] - 0s 653us/step - loss: 11.8765
Epoch 53/110
455/455 [==============================] - 0s 703us/step - loss: 11.6918
Epoch 54/110
455/455 [==============================] - 0s 624us/step - loss: 11.5367
Epoch 55/110
455/455 [==============================] - 0s 625us/step - loss: 11.5480
Epoch 56/110
455/455 [==============================] - 0s 501us/step - loss: 11.4138
Epoch 57/110
455/455 [==============================] - 0s 474us/step - loss: 11.3663
Epoch 58/110
455/455 [==============================] - 0s 499us/step - loss: 11.2216
Epoch 59/110
455/455 [==============================] - 0s 477us/step - loss: 11.0739
Epoch 60/110
455/455 [==============================] - 0s 537us/st

455/455 [==============================] - 0s 308us/step - loss: 15.9020
Epoch 35/110
455/455 [==============================] - 0s 307us/step - loss: 15.5943
Epoch 36/110
455/455 [==============================] - 0s 292us/step - loss: 15.2460
Epoch 37/110
455/455 [==============================] - 0s 301us/step - loss: 14.9184
Epoch 38/110
455/455 [==============================] - 0s 286us/step - loss: 14.5464
Epoch 39/110
455/455 [==============================] - 0s 307us/step - loss: 14.4192
Epoch 40/110
455/455 [==============================] - 0s 310us/step - loss: 14.1362
Epoch 41/110
455/455 [==============================] - 0s 296us/step - loss: 13.9263
Epoch 42/110
455/455 [==============================] - 0s 286us/step - loss: 13.7876
Epoch 43/110
455/455 [==============================] - 0s 350us/step - loss: 13.5138
Epoch 44/110
455/455 [==============================] - 0s 299us/step - loss: 13.4054
Epoch 45/110
455/455 [==============================] - 0s 299us/st

Epoch 3/110
456/456 [==============================] - 0s 401us/step - loss: 416.0332
Epoch 4/110
456/456 [==============================] - 0s 373us/step - loss: 245.0634
Epoch 5/110
456/456 [==============================] - 0s 442us/step - loss: 113.3324
Epoch 6/110
456/456 [==============================] - 0s 439us/step - loss: 62.4561
Epoch 7/110
456/456 [==============================] - 0s 424us/step - loss: 46.5795
Epoch 8/110
456/456 [==============================] - 0s 424us/step - loss: 37.7769
Epoch 9/110
456/456 [==============================] - 0s 447us/step - loss: 32.9887
Epoch 10/110
456/456 [==============================] - 0s 446us/step - loss: 30.5008
Epoch 11/110
456/456 [==============================] - 0s 433us/step - loss: 28.6579
Epoch 12/110
456/456 [==============================] - 0s 497us/step - loss: 27.7357
Epoch 13/110
456/456 [==============================] - 0s 436us/step - loss: 26.5012
Epoch 14/110
456/456 [==============================] - 0s

456/456 [==============================] - 0s 491us/step - loss: 6.2246
Epoch 84/110
456/456 [==============================] - 0s 447us/step - loss: 6.1859
Epoch 85/110
456/456 [==============================] - 0s 439us/step - loss: 6.1591
Epoch 86/110
456/456 [==============================] - 0s 447us/step - loss: 6.1418
Epoch 87/110
456/456 [==============================] - 0s 456us/step - loss: 6.1538
Epoch 88/110
456/456 [==============================] - 0s 447us/step - loss: 6.1090
Epoch 89/110
456/456 [==============================] - 0s 439us/step - loss: 6.0245
Epoch 90/110
456/456 [==============================] - 0s 456us/step - loss: 6.0835
Epoch 91/110
456/456 [==============================] - 0s 456us/step - loss: 6.0436
Epoch 92/110
456/456 [==============================] - 0s 447us/step - loss: 5.9815
Epoch 93/110
456/456 [==============================] - 0s 430us/step - loss: 5.9762
Epoch 94/110
456/456 [==============================] - 0s 447us/step - loss: 

456/456 [==============================] - 0s 456us/step - loss: 9.7404
Epoch 69/110
456/456 [==============================] - 0s 440us/step - loss: 9.7700
Epoch 70/110
456/456 [==============================] - 0s 430us/step - loss: 9.5221
Epoch 71/110
456/456 [==============================] - 0s 421us/step - loss: 9.6766
Epoch 72/110
456/456 [==============================] - 0s 421us/step - loss: 9.5969
Epoch 73/110
456/456 [==============================] - 0s 421us/step - loss: 9.5242
Epoch 74/110
456/456 [==============================] - 0s 456us/step - loss: 9.4401
Epoch 75/110
456/456 [==============================] - 0s 439us/step - loss: 9.4400
Epoch 76/110
456/456 [==============================] - 0s 439us/step - loss: 9.4271
Epoch 77/110
456/456 [==============================] - 0s 430us/step - loss: 9.3103
Epoch 78/110
456/456 [==============================] - 0s 456us/step - loss: 9.2709
Epoch 79/110
456/456 [==============================] - 0s 439us/step - loss: 

Since the model with rmsprop, 110 epochs, and two layers of 26 and 7 neurons produced the most efficient result so we will be using that.

# Step 7. Rewriting the code using the Keras Functional API
Now rewrite the code that you have written so far using the Keras Sequential API in Kearas Functional API.


In [43]:
import keras
seed = 7
np.random.seed(seed)


def create_model():
    inputs = keras.Input(shape = X[1].shape)
    x_1 = layers.Dense(26, activation = 'relu', kernel_initializer='normal')(inputs)
    x_2 = layers.Dense(7, activation = 'relu', kernel_initializer='normal')(x_1)
    outputs = layers.Dense(1, kernel_initializer='normal')(x_2)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error', metrics = ['mean_squared_error'])
    return model
  

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=create_model, epochs=110, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -20.33 (24.75) MSE


# Step 8. Rewriting the code by doing Model Subclassing
Now rewrite the code that you have written so far using the Keras Model Subclassing as mentioned in the Chollet April 9, 2018 presentation.

Reference:
https://www.tensorflow.org/api_docs/python/tf/keras/Model
Please note you will have to use TensorFlow 1.7+ with built-in Keras


In [45]:
import keras
seed = 7
np.random.seed(seed)

class MyModel(keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = layers.Dense(26, activation = 'relu', kernel_initializer = 'normal')
        self.dense2 = layers.Dense(7, activation = 'relu', kernel_initializer = 'normal')
        self.dense3 = layers.Dense(1, kernel_initializer = 'normal')
        
    def call(self, inputs):
        x_1 = self.dense1(inputs)
        x_2 = self.dense2(x_1)
        return self.dense3(x_2)
    
  
def create_model():
  model = MyModel()
  model.compile(optimizer = 'rmsprop',loss = 'mean_squared_error',metrics=['mean_squared_error'] )
  return model

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=create_model, epochs=110, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -20.33 (24.75) MSE


# Step 9. Rewriting the code without using scikit-learn
Once you have written the model in all three API style you are required to do k-fold cross validation without using scikit-learn library.



In [46]:
np.random.seed(seed)
data = dataframe.values
np.random.shuffle(data)
len(data)

506

In [16]:
X = data[:, 0:13]
Y = data[:, 13]

In [70]:
len(X.mean(axis = 0))

13

In [71]:
def model1():
    model = models.Sequential()
    model.add(layers.Dense(26, activation = 'relu',kernel_initializer = 'normal', input_shape = (X.shape[1],)))
    model.add(layers.Dense(7, activation = 'relu',kernel_initializer = 'normal'))
    model.add(layers.Dense(1, kernel_initializer = 'normal',))
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error', metrics = ['mean_squared_error'])
    return model


seed = 7
np.random.seed(seed)
k = 10
num_val_samples = len(X) // k #integer division
print(num_val_samples)
num_epochs = 110
results = []

for i in range(k):
    print("Processing field #", i)
    #prepairing the validation data:data from partition K
    val_data = X[i*num_val_samples:(i+1)*num_val_samples]      
    val_targets = Y[i*num_val_samples:(i+1)*num_val_samples]
    #prepairing training data from all other partitions
    partial_train_data = np.concatenate([
        X[:i*num_val_samples],
        X[(i+1)*num_val_samples:]
    ], axis = 0)
    partial_train_target = np.concatenate([
        Y[:i*num_val_samples],
        Y[(i+1)*num_val_samples:]
    ],axis = 0)
    partial_train_data_mean = partial_train_data.mean(axis = 0)
    partial_train_data_std = partial_train_data.std(axis = 0)
    partial_train_data = (partial_train_data - partial_train_data_mean)/partial_train_data_std
    val_data = (val_data - partial_train_data_mean)/partial_train_data_std
   
    model = model1()
    model.fit(partial_train_data, 
              partial_train_target,
              epochs = num_epochs, 
              batch_size = 5, verbose = 0)
    results.append(model.evaluate(val_data, val_targets))

50
Processing field # 0
50/50 [==============================] - 0s 7ms/step
Processing field # 1
50/50 [==============================] - 0s 7ms/step
Processing field # 2
50/50 [==============================] - 0s 8ms/step
Processing field # 3
50/50 [==============================] - 0s 8ms/step
Processing field # 4
50/50 [==============================] - 0s 8ms/step
Processing field # 5
50/50 [==============================] - 0s 9ms/step
Processing field # 6
50/50 [==============================] - 1s 10ms/step
Processing field # 7
50/50 [==============================] - 1s 10ms/step
Processing field # 8
50/50 [==============================] - 1s 10ms/step
Processing field # 9
50/50 [==============================] - 0s 9ms/step


In [59]:
model.metrics_names

['loss', 'mean_squared_error']

In [72]:
mse = []
for result in results:
    mse.append(result[1])
mse = np.asarray(mse)
overall_MSE = mse.mean()
overall_MSE

19.916297643661498